# PySimpleGUI

Python simple gui is a wrapper library for tkinter. it simplified some tedious process for simple tasks. Since I dont need advanced UI design, it is reasonable to use python simple gui to design gui for this bot


the following is an example of pysimplegui integrated with asyncio and aiojobs, which is how the bot is currently structured.

few important thing to note:
- while loop inside the gui function, can utilize scheduler.spawn() to run it concurrently along other tasks
- for this code, event is passed in order to control closing jobs and break out of the loop inside gui()
- somehow in the actual bot, that (timeout=1) inside window.read() is a must or it will block out the while loop and the entire event loop

In [6]:
import PySimpleGUI as sg
import asyncio
import aiojobs

# -- create layout and window for pysimplegui
layout = [
    [sg.Text("This is a discord bot")],
    [sg.Button("Shut Down")]
]
window = sg.Window(title="SlinkBot", layout= layout)

# -- asncio loop to read event
async def gui(event):
    while True:
        event, values = window.read(timeout=1)
        if event == "Shut Down" or event == sg.WIN_CLOSED:
            break

# -- main() create the global scheduler, event to pass in, and controlling spawn job and close job
async def main():
    scheduler = await aiojobs.create_scheduler()
    global job
    event = None
    sg.easy_print(layout)
    job = await scheduler.spawn(gui(event))
    await job.wait()
    if event == "Shut Down" or event == sg.WIN_CLOSED:
        await job.close()
        window.close()

await main()